In [1]:
import torch
from VidUtil.torch_utils import inspect_checkpoint

checkpoint_path = "/home/denninge/CamC2V/results/camc2v_camctrl_ctrlnet03/checkpoints/step=1.ckpt/checkpoint/mp_rank_00_model_states.pt"
checkpoint_info = inspect_checkpoint(checkpoint_path, return_dict=True)

/home/denninge/anaconda3/envs/cami2v/lib/python3.10/site-packages/torch/cuda/__init__.py:58: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/home/denninge/VidUtil/VidUtil/torch_utils/inspect_checkpoint.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are e

In [2]:
import torch
ckpt = torch.load(checkpoint_path, map_location='cpu')

/tmp/ipykernel_2793420/1512800985.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(checkpoint_path, map_location='cpu')


In [6]:
ckpt.keys()

dict_keys(['module', 'buffer_names', 'optimizer', 'param_shapes', 'frozen_param_shapes', 'shared_params', 'frozen_param_fragments', 'lr_scheduler', 'data_sampler', 'random_ltd', 'sparse_tensor_module_names', 'skipped_steps', 'global_steps', 'global_samples', 'dp_world_size', 'mp_world_size', 'ds_config', 'ds_version', 'epoch', 'global_step', 'pytorch-lightning_version', 'loops', 'callbacks', 'lr_schedulers'])

In [3]:
print(ckpt["module"]["model.diffusion_model.context_encoder.zero_convolutions.0.conv.weight"])

tensor([[[[[-9.5963e-06]]],


         [[[ 9.4771e-06]]],


         [[[-8.8215e-06]]],


         ...,


         [[[-9.5963e-06]]],


         [[[ 9.5367e-06]]],


         [[[-2.4438e-06]]]],



        [[[[ 8.6427e-06]]],


         [[[-9.2983e-06]]],


         [[[ 9.7752e-06]]],


         ...,


         [[[ 7.8082e-06]]],


         [[[ 9.7156e-06]]],


         [[[ 9.8348e-06]]]],



        [[[[ 8.5831e-06]]],


         [[[ 8.9407e-06]]],


         [[[-9.7752e-06]]],


         ...,


         [[[-9.1791e-06]]],


         [[[-9.3579e-06]]],


         [[[ 7.4506e-06]]]],



        ...,



        [[[[-8.4639e-06]]],


         [[[-9.5963e-06]]],


         [[[ 9.6560e-06]]],


         ...,


         [[[ 9.2387e-06]]],


         [[[-9.2983e-06]]],


         [[[ 9.4771e-06]]]],



        [[[[ 9.2983e-06]]],


         [[[-9.3579e-06]]],


         [[[-8.6427e-06]]],


         ...,


         [[[ 9.6560e-06]]],


         [[[-8.8811e-06]]],


         [[[ 9.4175e-06]]]

In [9]:
for k in ckpt["module"].keys():
    print(k)

betas
alphas_cumprod
alphas_cumprod_prev
sqrt_alphas_cumprod
sqrt_one_minus_alphas_cumprod
log_one_minus_alphas_cumprod
sqrt_recip_alphas_cumprod
sqrt_recipm1_alphas_cumprod
posterior_variance
posterior_log_variance_clipped
posterior_mean_coef1
posterior_mean_coef2
model.diffusion_model.time_embed.0.weight
model.diffusion_model.time_embed.0.bias
model.diffusion_model.time_embed.2.weight
model.diffusion_model.time_embed.2.bias
model.diffusion_model.fps_embedding.0.weight
model.diffusion_model.fps_embedding.0.bias
model.diffusion_model.fps_embedding.2.weight
model.diffusion_model.fps_embedding.2.bias
model.diffusion_model.input_blocks.0.0.weight
model.diffusion_model.input_blocks.0.0.bias
model.diffusion_model.input_blocks.1.0.in_layers.0.weight
model.diffusion_model.input_blocks.1.0.in_layers.0.bias
model.diffusion_model.input_blocks.1.0.in_layers.2.weight
model.diffusion_model.input_blocks.1.0.in_layers.2.bias
model.diffusion_model.input_blocks.1.0.emb_layers.1.weight
model.diffusion_m

In [8]:
checkpoint_info.keys()

dict_keys(['metadata', 'state_dict', 'totals', 'options'])

In [9]:
parameter_names = list(checkpoint_info['state_dict'].keys())

In [14]:
import re
from collections import defaultdict
from pprint import pprint

def extract_input_block_layers(keys, start_idx=0, end_idx=8, root="model.diffusion_model.middle_block"):
    """
    Extract parameters that live under model.diffusion_model.input_blocks.{i}
    for i in [start_idx, end_idx] inclusive.

    Returns a nested dict:
    {
      i: {
        "all": [full_param_key, ...],                      # all raw keys for this block
        "by_submodule": { submodule: [keys...], ... },     # grouped by the token right after input_blocks.i
        "unique_submodules": {submodule, ...}              # convenience set
      },
      ...
    }
    """
    results = {}
    # Precompile regex: start of string, root, dot, (i), dot, capture next token (no dot), then the rest
    # Example match groups for "model.diffusion_model.input_blocks.2.0.emb_layers.1.bias":
    #   block_i = "2", submodule = "0", remainder = "emb_layers.1.bias"
    pattern = re.compile(rf"^{re.escape(root)}\.(\d+)\.([^\.]+)(?:\.(.*))?$")

    # Prepare containers for each i
    for i in range(start_idx, end_idx + 1):
        results[i] = {
            "all": [],
            "by_submodule": defaultdict(list),
            "unique_submodules": set(),
        }

    for k in keys:
        m = pattern.match(k)
        if not m:
            continue

        block_i_str, submodule, remainder = m.groups()
        try:
            block_i = int(block_i_str)
        except ValueError:
            continue

        # Keep only requested range
        if block_i < start_idx or block_i > end_idx:
            continue

        # Store
        results[block_i]["all"].append(k)
        results[block_i]["by_submodule"][submodule].append(k)
        results[block_i]["unique_submodules"].add(submodule)

    # Turn defaultdicts into plain dicts for cleanliness
    for i in range(start_idx, end_idx + 1):
        results[i]["by_submodule"] = dict(results[i]["by_submodule"])

    return results

# --------- Example usage ----------
blocks = extract_input_block_layers(parameter_names, start_idx=0, end_idx=8)

# Print a concise summary:
for i in range(0, 9):
    print(f"\n=== input_blocks.{i} ===")
    print(f"Total params: {len(blocks[i]['all'])}")
    with open(f"../model_architecture/middle_block_{i}.txt", "w") as f:
        for k in blocks[i]["all"]:
            f.write(k + "\n")
    print("Submodules:", sorted(blocks[i]["unique_submodules"]))


=== input_blocks.0 ===
Total params: 26
Submodules: ['emb_layers', 'in_layers', 'out_layers', 'temopral_conv']

=== input_blocks.1 ===
Total params: 29
Submodules: ['norm', 'proj_in', 'proj_out', 'transformer_blocks']

=== input_blocks.2 ===
Total params: 34
Submodules: ['norm', 'proj_in', 'proj_out', 'transformer_blocks']

=== input_blocks.3 ===
Total params: 26
Submodules: ['emb_layers', 'in_layers', 'out_layers', 'temopral_conv']

=== input_blocks.4 ===
Total params: 0
Submodules: []

=== input_blocks.5 ===
Total params: 0
Submodules: []

=== input_blocks.6 ===
Total params: 0
Submodules: []

=== input_blocks.7 ===
Total params: 0
Submodules: []

=== input_blocks.8 ===
Total params: 0
Submodules: []
